In [103]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import json

import lxml.html as lh
!pip install folium
import folium
print('Libraries imported.')

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


In [104]:
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [105]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [106]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A
5,Downtown Toronto,Regent Park\n,M5A
6,North York,Lawrence Heights\n,M6A
7,North York,Lawrence Manor\n,M6A
8,Queen's Park,Not assigned\n,M7A
9,Not assigned,Not assigned\n,M8A


In [9]:
df.columns = df.columns.str.strip()
df['Neighbourhood'] = df['Neighbourhood'].str.replace('/n', '')

In [10]:
df.columns = df.columns.str.strip()

In [11]:
df['Neighbourhood']

0                                         Not assigned\n
1                                         Not assigned\n
2                                            Parkwoods\n
3                                     Victoria Village\n
4                                         Harbourfront\n
5                                          Regent Park\n
6                                     Lawrence Heights\n
7                                       Lawrence Manor\n
8                                         Not assigned\n
9                                         Not assigned\n
10                                    Islington Avenue\n
11                                               Rouge\n
12                                             Malvern\n
13                                        Not assigned\n
14                                     Don Mills North\n
15                                    Woodbine Gardens\n
16                                       Parkview Hill\n
17                             

In [12]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n', '')

In [13]:
df = df[['Postcode','Borough','Neighbourhood']]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [14]:

df = df.drop("Not assigned", axis=0)

ValueError: labels ['Not assigned'] not contained in axis

In [15]:
df= df.set_index("Postcode")

In [16]:
df.reset_index(level=0, inplace=True)

In [17]:
df=  df[['Postcode','Borough','Neighbourhood']]

In [18]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n', '')

In [19]:
df2.columns = df2.columns.str.strip()

NameError: name 'df2' is not defined

In [20]:
df = df[df.Neighbourhood != 'Not assigned']

In [21]:
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [22]:
df2=df
df2

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [23]:
g=df2.groupby('Postcode').agg("Neighbourhood")
#t=pd.DataFrame(g)
#t
g

### tmp is a condition to keep the value of the first column and append the value of the second one

In [24]:
def tmp(group_series):
    if (group_series==group_series.iloc[0]).all():
        return (group_series.iloc[0])
    else:
        return (group_series.sum())
x = df2.groupby('Postcode').agg(tmp)

In [25]:
z=x.reset_index()

In [26]:
z.shape

(102, 3)

In [27]:
loc = pd.read_csv('http://cocl.us/Geospatial_data')
loc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [28]:
result = pd.concat([z, loc], axis=1, join='inner')
result

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,M1B,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,M1C,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,East Birchmount ParkIonviewKennedy Park,M1K,43.727929,-79.262029
7,M1L,Scarborough,ClairleaGolden MileOakridge,M1L,43.711112,-79.284577
8,M1M,Scarborough,CliffcrestCliffsideScarborough Village West,M1M,43.716316,-79.239476
9,M1N,Scarborough,Birch CliffCliffside West,M1N,43.692657,-79.264848


In [29]:
final = result.drop(['Postal Code'],axis=1)
final

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,East Birchmount ParkIonviewKennedy Park,43.727929,-79.262029
7,M1L,Scarborough,ClairleaGolden MileOakridge,43.711112,-79.284577
8,M1M,Scarborough,CliffcrestCliffsideScarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch CliffCliffside West,43.692657,-79.264848


In [30]:
map_manhat = folium.Map(location=[43.806686, -79.194353], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhat)  
    
map_manhat

### define foursquare


In [31]:
CLIENT_ID = 'VYYCVYU2HWX3GT32SS5L5MKEZJIYFJX2NQA4FM1GVPO5ABHP' # your Foursquare ID
CLIENT_SECRET = 'WX5CTWB3X1HXAFCWRGGOH40C3ZVAVXHL1420ARZ325XE0TX0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

## define functions that helps explore the dataframe

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
manhattan_venues = getNearbyVenues(names=final['Neighbourhood'],
                                   latitudes=final['Latitude'],
                                   longitudes=final['Longitude']
                                  )


RougeMalvern
Highland CreekRouge HillPort Union
GuildwoodMorningsideWest Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount ParkIonviewKennedy Park
ClairleaGolden MileOakridge
CliffcrestCliffsideScarborough Village West
Birch CliffCliffside West
Dorset ParkScarborough Town CentreWexford Heights
MaryvaleWexford
Agincourt
Clarks CornersSullivanTam O'Shanter
Agincourt NorthL'Amoreaux EastMillikenSteeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
FairviewHenry FarmOriole
Bayview Village
Silver HillsYork Mills
NewtonbrookWillowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon ParkDon Mills South
Bathurst ManorDownsview NorthWilson Heights
Northwood ParkYork University
CFB TorontoDownsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine GardensParkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth WestRiverdale
The Beaches WestIndia Bazaar
Studio District
Lawrenc

In [35]:
print(manhattan_venues.shape)
manhattan_venues.head()

(2255, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,RougeMalvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,RougeMalvern,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Highland CreekRouge HillPort Union,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,Highland CreekRouge HillPort Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,GuildwoodMorningsideWest Hill,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [36]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AdelaideKingRichmond,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt NorthL'Amoreaux EastMillikenSteeles East,3,3,3,3,3,3
Albion GardensBeaumond HeightsHumbergateJamestownMount OliveSilverstoneSouth SteelesThistletown,4,4,4,4,4,4
AlderwoodLong Branch,15,15,15,15,15,15
Bathurst ManorDownsview NorthWilson Heights,18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
Bedford ParkLawrence Manor East,25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [37]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women'

In [38]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women'

In [39]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AdelaideKingRichmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2                  Bar  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3  Chinese Restaurant  0.25
4         Yoga Studio  0.00


----Agincourt NorthL'Amoreaux EastMillikenSteeles East----
                       venue  freq
0                 Playground  0.33
1           Asian Restaurant  0.33
2                       Park  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion GardensBeaumond HeightsHumbergateJamestownMount OliveSilverstoneSouth SteelesThistletown----
                       venue  freq
0                Pizza Place  0.25
1                   Bus Line  0.25
2                       Park  0.25
3          Mobile Phone Shop  0.25
4  Middle Eastern Restaurant  0.00


----AlderwoodLong Br

                        venue  freq
0                      Bakery   0.2
1  Construction & Landscaping   0.2
2                        Park   0.2
3               Deli / Bodega   0.2
4            Basketball Court   0.2


----East Birchmount ParkIonviewKennedy Park----
               venue  freq
0   Department Store  0.25
1         Playground  0.25
2        Coffee Shop  0.25
3     Discount Store  0.25
4  Mobile Phone Shop  0.00


----East Toronto----
                venue  freq
0         Coffee Shop  0.33
1                Park  0.33
2   Convenience Store  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----EmeryHumberlea----
                   venue  freq
0            Pizza Place  0.33
1    Empanada Restaurant  0.33
2               Pharmacy  0.33
3  Performing Arts Venue  0.00
4                   Park  0.00


----FairviewHenry FarmOriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.09
2           Coffee Shop  0.08
3            Re

                             venue  freq
0                       Playground   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Silver HillsYork Mills----
                       venue  freq
0                  Cafeteria   1.0
1                Yoga Studio   0.0
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----St. James Town----
         venue  freq
0         Café  0.06
1  Coffee Shop  0.06
2   Restaurant  0.05
3        Hotel  0.05
4    Gastropub  0.03


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.04
2          Restaurant  0.04
3        Cocktail Bar  0.03
4  Italian Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2  American Restaurant  0.05
3       

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AdelaideKingRichmond,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Bar,Hotel,Cosmetics Shop,Gym,Burger Joint
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Department Store
2,Agincourt NorthL'Amoreaux EastMillikenSteeles ...,Park,Asian Restaurant,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,Albion GardensBeaumond HeightsHumbergateJamest...,Park,Pizza Place,Mobile Phone Shop,Bus Line,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
4,AlderwoodLong Branch,Gym,Hobby Shop,Café,Mexican Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place,Bakery,American Restaurant,Fast Food Restaurant
5,Bathurst ManorDownsview NorthWilson Heights,Coffee Shop,Shopping Mall,Bridal Shop,Supermarket,Middle Eastern Restaurant,Deli / Bodega,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
6,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
7,Bedford ParkLawrence Manor East,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Butcher,Pharmacy,Restaurant,Cupcake Shop,Sushi Restaurant,Juice Bar,Ice Cream Shop
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Farmers Market,Steakhouse,Café,Cheese Shop,Beer Bar,Italian Restaurant
9,Birch CliffCliffside West,General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [42]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 3, 0, 0, 4, 0, 0, 0], dtype=int32)

In [65]:
# add clustering labels

#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#manhattan_merged = final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(c, on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,RougeMalvern,43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
1,M1C,Scarborough,Highland CreekRouge HillPort Union,43.784535,-79.160497,0.0,Bar,Construction & Landscaping,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,43.763573,-79.188711,0.0,Electronics Store,Spa,Pizza Place,Intersection,Tech Startup,Breakfast Spot,Mexican Restaurant,Rental Car Location,Medical Center,Women's Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Convenience Store,Korean Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Lounge,Hakka Restaurant,Bakery,Bank,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Dog Run,Diner


In [64]:
manhattan_merged = manhattan_merged.rename(index=str, columns={"Neighbourhood": "Neighborhood"})
manhattan_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,East Birchmount ParkIonviewKennedy Park,43.727929,-79.262029
7,M1L,Scarborough,ClairleaGolden MileOakridge,43.711112,-79.284577
8,M1M,Scarborough,CliffcrestCliffsideScarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch CliffCliffside West,43.692657,-79.264848


In [59]:
c = neighborhoods_venues_sorted.set_index('Neighborhood')
c

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,
AdelaideKingRichmond,0,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Bar,Hotel,Cosmetics Shop,Gym,Burger Joint
Agincourt,0,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Department Store
Agincourt NorthL'Amoreaux EastMillikenSteeles East,2,Park,Asian Restaurant,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
Albion GardensBeaumond HeightsHumbergateJamestownMount OliveSilverstoneSouth SteelesThistletown,3,Park,Pizza Place,Mobile Phone Shop,Bus Line,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
AlderwoodLong Branch,0,Gym,Hobby Shop,Café,Mexican Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place,Bakery,American Restaurant,Fast Food Restaurant
Bathurst ManorDownsview NorthWilson Heights,0,Coffee Shop,Shopping Mall,Bridal Shop,Supermarket,Middle Eastern Restaurant,Deli / Bodega,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
Bayview Village,4,Chinese Restaurant,Café,Japanese Restaurant,Bank,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
Bedford ParkLawrence Manor East,0,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Butcher,Pharmacy,Restaurant,Cupcake Shop,Sushi Restaurant,Juice Bar,Ice Cream Shop
Berczy Park,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Farmers Market,Steakhouse,Café,Cheese Shop,Beer Bar,Italian Restaurant


In [71]:
latitude = 43.781638
longitude = -79.239476

In [113]:

manhattan_merged

0      0
1      0
2      0
3      0
4      0
5      2
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     2
15     0
17     0
18     0
19     4
20     0
22     0
23     3
24     0
25     3
26     0
27     0
28     0
29     0
30     3
31     0
32     0
33     0
34     0
35     0
36     0
37     0
38     0
39     0
40     3
41     0
42     0
43     0
44     3
45     0
46     0
47     0
48     2
49     0
50     3
51     0
52     0
53     0
54     0
55     0
56     0
57     0
58     0
59     0
60     0
61     0
62     0
63     0
64     3
65     0
66     0
67     0
68     0
69     0
70     0
71     0
72     3
73     0
74     3
75     0
76     0
77     0
78     0
79     3
80     0
81     0
82     0
83     0
84     0
85     0
86     0
87     0
88     0
89     0
90     3
91     1
92     0
94     4
95     0
96     0
97     1
98     3
99     0
100    3
101    0
Name: Cluster Labels, dtype: int64

In [110]:
# create map
map_clusters = folium.Map(location=[43.806686, -79.194353], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip( manhattan_merged['Neighborhood'], manhattan_merged['Latitude'], manhattan_merged['Longitude'],  manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Neighborhood'

In [86]:
cluster


0.0